In [1]:
from nifstd.development.sparc.sheets import *


class MCSZSchema(SheetPlus):
    
    def get_sub_header_indexes(self, start_index, column):
        ''' Parse for bold text and returns its coordinates '''
        for sheet in self.grid['sheets']:
            if sheet['properties']['title'] == self.sheet_name:
                for datum in sheet['data']:
                    for i, row in enumerate(datum['rowData'][start_index:], start_index):
                        if 'values' in row:
                            try:
                                cell = row['values'][column]
                            except:
                                continue
                            if 'effectiveFormat' in cell:
                                if 'textFormat' in cell['effectiveFormat']:
                                    if cell['effectiveFormat']['textFormat'].get('bold'):
                                        if 'userEnteredValue' in cell:
                                            if cell['userEnteredValue'].get('stringValue'):
                                                yield i # , cell['userEnteredValue'].get('stringValue')
    
    def populate_graph(self):
        ''' Parse sheet for values "List[str]" with bold headers as keys "str" '''
        
        # Get Header & Sub-Header Indexes
        # Get Header Index
        for index, header in enumerate(self.raw_values):
            if header:
                headers_index = index
                break

        # populate reference headers to keep simplicity 
        # populate dictionary with keys
        # start from leaf header + 1 and 
                
        # Find Sub-Headers and fill in graph accordingly
        for column, header in enumerate(self.raw_values[headers_index]):
            header = self.clean(header)
            sub_header_indexes = list(self.get_sub_header_indexes(
                start_index = headers_index + 1,
                column = column
            ))
            for index, row in enumerate(self.values[headers_index + 1:]):
                sub_header = row[column]
                print(sub_header)
                sub_header = self.clean(sub_header)
                if not sub_header:
                    continue
                try:
                    sub_header_index = sub_header_indexes[sub_header_indexes.index(index)]
                except:
                    continue
                if sub_header_index:
                    try:
                        end_index = sub_header_indexes[sub_header_indexes.index(index) + 1] - 1
                    except:
                        end_index = None
                    sub_terms_list = self.get_sub_column(
                        start_index = sub_header_index + 1,
                        end_index =  end_index,
                        column = column,
                    )
                    self.tree[self.name][self.sheet_name][header][sub_header] = sub_terms_list

class Brainstem(MCSZSchema):
    name = 'uberon-spinal-parcellation'
    sheet_name = 'Brainstem'
    fetch_grid = True
    
brainstem = Brainstem()
#brainstem.build()
#brainstem.populate_graph()
# brainstem.build()
# brainstem.raw_values

[2019-07-08 09:25:45,224] -  WARNING - pyontutils.sheets -        sheets.py:176  - Sheet has malformed header, not setting byCol


In [4]:
from nifstd.development.sparc.sheets import open_custom_sparc_view_yml, YML_DELIMITER, YML_DISPLAY_DELIMITER
from collections import OrderedDict
from pyontutils.config import devconfig
from pathlib import Path
import yaml

def ordered_load(stream, Loader=yaml.SafeLoader, object_pairs_hook=OrderedDict):
    class OrderedLoader(Loader):
        pass
    def construct_mapping(loader, node):
        loader.flatten_mapping(node)
        return object_pairs_hook(loader.construct_pairs(node))
    OrderedLoader.add_constructor(
        yaml.resolver.BaseResolver.DEFAULT_MAPPING_TAG,
        construct_mapping)
    return yaml.load(stream, OrderedLoader)

with open(Path(devconfig.resources)/'sparc_term_versions/sparc_terms2-mod.txt', 'rt') as infile:
    raw_yaml = ''
    visited = {}
    for line in infile.readlines()[1:]:
        # last line doesnt have newline so we cant just replace it
        raw_yaml += line.replace('\n', '').replace(YML_DISPLAY_DELIMITER, YML_DELIMITER) + ':\n'
        label, *curies = line.replace('\n', '').split('\t')
        if visited.get(label):
            print('SHIT', label)
        else:
            visited[label] = True
    yaml = ordered_load(raw_yaml)
        
print(label, curies)

SHIT     abducens nerve
SHIT     accessory XI nerve cranial component
SHIT     accessory XI nerve
SHIT     anterior lateral line nerve
SHIT     anterodorsal lateral line nerve (ADLLN)
SHIT     anteroventral lateral line nerve (AVLLN)
SHIT     chorda tympani branch of facial nerve
SHIT     cranial nerve
SHIT     dorsal anterior lateral line nerve
SHIT     facial nerve
SHIT     glossopharyngeal nerve
SHIT     hypoglossal nerve
SHIT     jugal ramule
SHIT     lateral line nerve
SHIT     middle lateral line nerve (MLLN)
SHIT     middle lateral line nerve
SHIT     oculomotor nerve
SHIT     olfactory nerve
SHIT     oral ramule
SHIT     posterior lateral line nerve (PLLN)
SHIT     posterior lateral line nerve
SHIT     preopercular ramule
SHIT     ramulus suprabranchialis anterior
SHIT     ramulus suprabranchialis posterior
SHIT     ramus auricularis of the vagus nerve
SHIT     ramus buccal
SHIT     ramus lateral
SHIT     ramus mandibularis externus
SHIT     ramus palatinus
SHIT     ramus super

In [19]:
nx.DiGraph(yaml).nodes

NodeView(('Cranial NervesὊ9UBERON:0001785', 'Nerves (Also Includes Cranial)', 'Nerve roots of spinal cord segments', 'UBERON', 'Nieuwenhuys', 'Atlas Nomenclature', 'Peripheral Nervous SystemὊ9UBERON:0000010', 'OrganList (MBF)', 'FlatMap', 'SparcILX', 'Minimal information model (MIS)', 'ApiNATOMY', 'Abducens nerveὊ9UBERON:0001646', 'Accessory nerve', 'Facial nerveὊ9UBERON:0001647', 'Glossopharyngeal nerveὊ9UBERON:0001649', 'Hypoglossal nerveὊ9UBERON:0001650', 'Oculomotor nerveὊ9UBERON:0001643', 'Olfactory nerveὊ9UBERON:0001579', 'Terminal nerveὊ9UBERON:0002924', 'Trigeminal nerveὊ9UBERON:0001645', 'Trochlear nerveὊ9UBERON:0001644', 'Vagus nerveὊ9UBERON:0001759', 'Vestibulocochlear nerveὊ9UBERON:0001648', 'Vomeronasal nerveὊ9UBERON:0009121', 'abducens nerveὊ9UBERON:0001646', 'accessory XI nerve cranial componentὊ9UBERON:0009673', 'accessory XI nerveὊ9UBERON:0002019', 'anterior lateral line nerveὊ9UBERON:2000425', 'anterodorsal lateral line nerve (ADLLN)Ὂ9UBERON:3010105', 'anteroventral l

In [41]:
from pyontutils.core import OntGraph
OntGraph()

In [9]:
dict_ = open_custom_sparc_view_yml(False)
print(dict_.keys())
tree = nx.DiGraph(dict_)
nx.bfs_tree(tree, 'Cranial NervesὊ9UBERON:0001785').edges()

odict_keys(['Cranial NervesὊ9UBERON:0001785', 'Nerves (Also Includes Cranial)', 'Nerve roots of spinal cord segments', 'UBERON', 'Nieuwenhuys', 'Atlas Nomenclature', 'Peripheral Nervous SystemὊ9UBERON:0000010', 'OrganList (MBF)', 'FlatMap', 'SparcILX', 'Minimal information model (MIS)', 'ApiNATOMY'])


OutEdgeView([('Cranial NervesὊ9UBERON:0001785', 'Abducens nerveὊ9UBERON:0001646'), ('Cranial NervesὊ9UBERON:0001785', 'Accessory nerve'), ('Cranial NervesὊ9UBERON:0001785', 'Facial nerveὊ9UBERON:0001647'), ('Cranial NervesὊ9UBERON:0001785', 'Glossopharyngeal nerveὊ9UBERON:0001649'), ('Cranial NervesὊ9UBERON:0001785', 'Hypoglossal nerveὊ9UBERON:0001650'), ('Cranial NervesὊ9UBERON:0001785', 'Oculomotor nerveὊ9UBERON:0001643'), ('Cranial NervesὊ9UBERON:0001785', 'Olfactory nerveὊ9UBERON:0001579'), ('Cranial NervesὊ9UBERON:0001785', 'Terminal nerveὊ9UBERON:0002924'), ('Cranial NervesὊ9UBERON:0001785', 'Trigeminal nerveὊ9UBERON:0001645'), ('Cranial NervesὊ9UBERON:0001785', 'Trochlear nerveὊ9UBERON:0001644'), ('Cranial NervesὊ9UBERON:0001785', 'Vagus nerveὊ9UBERON:0001759'), ('Cranial NervesὊ9UBERON:0001785', 'Vestibulocochlear nerveὊ9UBERON:0001648'), ('Cranial NervesὊ9UBERON:0001785', 'Vomeronasal nerveὊ9UBERON:0009121'), ('Cranial NervesὊ9UBERON:0001785', 'abducens nerveὊ9UBERON:0001646')

In [28]:
import networkx as nx
from pyontutils.core import OntId, OntTerm, sgr, OntGraph
from rdflib import *
from ontquery.plugins.services import SciGraphRemote
import os
from nifstd.development.sparc.sheets import open_custom_sparc_view_yml, YML_DELIMITER

# node = nx.DiGraph(yaml).neighbors('vagus nerveὊ9UBERON:0001759')
# iri = OntId('UBERON:0001759').iri
# qr = list(OntTerm.query(iri=iri))[0]
# print(qr)
g = OntGraph()
sgr.setup(instrumented=OntTerm)
def get_triples_from_dict(dict_):
    triples = []
    for node in nx.DiGraph(dict_).nodes:
        label, *curies = node.split(YML_DELIMITER)
        print(label, curies)
        if curies:
            for curie in curies:
                iri = OntId(curie).iri
                for key, value in sgr.sgg.getNode(iri)['nodes'][0]['meta'].items():
                    if key.startswith('http://') or key.startswith('https://'):
                        if not value:
                            continue
                        elif not isinstance(value, list):
                            value = [value]
                        for v in value:
                            if isinstance(v, URIRef) or isinstance(v, Literal) or isinstance(v, BNode):
                                pass
                            elif v.startswith('http://') or v.startswith('https://'):
                                v = URIRef(v)
                            else:
                                v = Literal(v)
                            triples.append((URIRef(iri), URIRef(key), v))
    return []

dict_ = open_custom_sparc_view_yml(False)
print(list(dict_.values())[0])
get_triples_from_dict(dict_)

OrderedDict([('Abducens nerveὊ9UBERON:0001646', None), ('Accessory nerve', None), ('Facial nerveὊ9UBERON:0001647', None), ('Glossopharyngeal nerveὊ9UBERON:0001649', None), ('Hypoglossal nerveὊ9UBERON:0001650', None), ('Oculomotor nerveὊ9UBERON:0001643', None), ('Olfactory nerveὊ9UBERON:0001579', None), ('Terminal nerveὊ9UBERON:0002924', None), ('Trigeminal nerveὊ9UBERON:0001645', None), ('Trochlear nerveὊ9UBERON:0001644', None), ('Vagus nerveὊ9UBERON:0001759', None), ('Vestibulocochlear nerveὊ9UBERON:0001648', None), ('Vomeronasal nerveὊ9UBERON:0009121', None), ('abducens nerveὊ9UBERON:0001646', None), ('accessory XI nerve cranial componentὊ9UBERON:0009673', None), ('accessory XI nerveὊ9UBERON:0002019', None), ('anterior lateral line nerveὊ9UBERON:2000425', None), ('anterodorsal lateral line nerve (ADLLN)Ὂ9UBERON:3010105', None), ('anteroventral lateral line nerve (AVLLN)Ὂ9UBERON:3010109', None), ('chorda tympani branch of facial nerveὊ9UBERON:0009675', None), ('cranial nerveὊ9UBERON:0

http://www.ncbi.nlm.nih.gov/pubmed/2286018 state: 'The presence of an additional cranial nerve (the nervus terminalis or cranial nerve zero) is well documented in many non-human vertebrate species. However, its existence in the adult human has been disputed. The present study focused on the structure and incidence of this nerve in the adult human brain. The nerve was examined post-mortem in 10 adult brains using dissection microscopy, light microscopy, transmission electron microscopy, and immunohistochemistry. In all specimens, the nervus terminalis was identified bilaterally as a microscopic plexus of unmyelinated peripheral nerve fascicles in the subarachnoid space covering the gyrus rectus of the orbital surface of the frontal lobes. The plexus appeared in the region of the cribriform plate of the ethmoid and coursed posteriorly to the vicinity of the olfactory trigone, medial olfactory gyrus, and lamina terminalis. We conclude that the terminal nerve is a common finding in the adu

Trigeminal nerve ['UBERON:0001645']
Trochlear nerve ['UBERON:0001644']
Vagus nerve ['UBERON:0001759']
Vestibulocochlear nerve ['UBERON:0001648']
Vomeronasal nerve ['UBERON:0009121']
abducens nerve ['UBERON:0001646']
accessory XI nerve cranial component ['UBERON:0009673']
accessory XI nerve ['UBERON:0002019']
anterior lateral line nerve ['UBERON:2000425']
anterodorsal lateral line nerve (ADLLN) ['UBERON:3010105']
anteroventral lateral line nerve (AVLLN) ['UBERON:3010109']
chorda tympani branch of facial nerve ['UBERON:0009675']
cranial nerve ['UBERON:0001785']
dorsal anterior lateral line nerve ['UBERON:2001480']
facial nerve ['UBERON:0001647']
glossopharyngeal nerve ['UBERON:0001649']
hypoglossal nerve ['UBERON:0001650']
jugal ramule ['UBERON:3010778']
lateral line nerve ['UBERON:0008906']
middle lateral line nerve (MLLN) ['UBERON:3010126']
middle lateral line nerve ['UBERON:2001482']
oculomotor nerve ['UBERON:0001643']
olfactory nerve ['UBERON:0001579']
oral ramule ['UBERON:3010794']


http://www.ncbi.nlm.nih.gov/pubmed/2286018 state: 'The presence of an additional cranial nerve (the nervus terminalis or cranial nerve zero) is well documented in many non-human vertebrate species. However, its existence in the adult human has been disputed. The present study focused on the structure and incidence of this nerve in the adult human brain. The nerve was examined post-mortem in 10 adult brains using dissection microscopy, light microscopy, transmission electron microscopy, and immunohistochemistry. In all specimens, the nervus terminalis was identified bilaterally as a microscopic plexus of unmyelinated peripheral nerve fascicles in the subarachnoid space covering the gyrus rectus of the orbital surface of the frontal lobes. The plexus appeared in the region of the cribriform plate of the ethmoid and coursed posteriorly to the vicinity of the olfactory trigone, medial olfactory gyrus, and lamina terminalis. We conclude that the terminal nerve is a common finding in the adu

terminal nerve ['UBERON:0002924']
trigeminal nerve ['UBERON:0001645']
trochlear nerve ['UBERON:0001644']
vagus nerve ['UBERON:0001759']
ventral anterior lateral line nerve ['UBERON:2001481']
vestibulocochlear nerve ['UBERON:0001648']
vomeronasal nerve ['UBERON:0009121']
abdomen nerve ['UBERON:0003429']
accessory XI nerve spinal component ['UBERON:0009674']
ampullary nerve ['UBERON:2005144']
ankle nerve ['UBERON:0003446']
ansa cervicalis ['UBERON:0005430']
anterior superior alveolar nerve ['UBERON:0035646']
arm nerve ['UBERON:0003433']
autonomic nerve ['UBERON:0034728']
axillary nerve ['UBERON:0001493']
back nerve ['UBERON:0004215']
buccal nerve ['UBERON:0013646']
carotid sinus nerve ['UBERON:0009009']
chest nerve ['UBERON:0003432']
cholinergic enteric nerve ['UBERON:0010406']
coccygeal nerve ['UBERON:0009629']
cochlear nerve ['UBERON:0004727']
common fibular nerve ['UBERON:0001324']
communicating branch of spinal nerve ['UBERON:0017642']
deep fibular nerve ['UBERON:0035207']
deep tempo

BadCurieError: Could not split curie 'Primary sensory and Primary Motor Nuclei' is it actually an identifier?

In [34]:
OntTerm??

Init signature:
OntTerm(
    curie_or_iri=None,
    label=None,
    term=None,
    search=None,
    validated=None,
    query=None,
    **kwargs,
)
Docstring:      classes that instrument a type of identifier to make it actionable 
Source:        
class OntTerm(oq.OntTerm, OntId):
    def atag(self, curie=False, **kwargs):
        return hfn.atag(self.iri, self.curie if curie else self.label, **kwargs)  # TODO schema.org ...
File:           ~/Dropbox/git/pyontutils/pyontutils/core.py
Type:           type
Subclasses:     IlxTerm


In [73]:
from ontologysoup import open_ontology
uberon = '/home/tmsincomb/Dropbox/git/NIF-Ontology/ttl/external/uberon.owl'
uberon = '/home/tmsincomb/Dropbox/git/NIF-Ontology/ttl/external/uberon.owl'
uberon = '/home/tmsincomb/Dropbox/git/NIF-Ontology/ttl/external/uberon.owl'
open_ontology()

Signature:
open_ontology(
    graph: Union[list, rdflib.graph.Graph, str],
    extra_prefixes: Union[List[tuple], dict] = {},
) -> rdflib.graph.Graph
Source:   
def open_ontology(graph:Union[list, Graph, str], extra_prefixes:Union[List[tuple], dict]={}) -> Graph:
    ''' Opens rdflib.Graph()s or data/files that can be converted to rdflib.Graph()s
        Args:
            graph:
                rdflib Graph, a path to rdflib Graph that could be picklized, or a list of the previous.
                Could also just be a list of triples.
            extra_prefixes: prefixes you may want to add bc graph is not complete.
        Return rdflib Graph '''
    return OpenOntology(graph_obj_to_open=graph, extra_prefixes=extra_prefixes).graph
File:      ~/Dropbox/git/ontologysoup/ontologysoup/opener.py
Type:      function


In [24]:
list(node)

[]

In [35]:
from nifstd.development.sparc.sheets import open_custom_sparc_view_yml
from typing import *
import networkx as nx
yml = open_custom_sparc_view_yml()

def class_hierarchy(dag:List[tuple], descending=True) -> list:
    ''' Topological Sorting 
    Args: 
        dag: directed acyclic graph that is a mappings of list of tuples with len of 2
    Returns:
        Ordered list of single entities in topological ordering of choice
    Examples:
        >>> class_hierarchy([(1, 2), (2, 3)]) 
        [3, 2, 1]
    '''
    dag = nx.DiGraph(dag) 
    dag = list(nx.topological_sort(dag))
    if descending:
        dag = list(reversed(dag))
    return dag

dag = nx.DiGraph(yml)

In [69]:
from pyontutils import scigraph
scigraph.Dynamic('https://scicrunch.org/api/1/sparc-scigraph')

In [66]:
import ontquery as oq
from pyontutils.core import OntTerm
SGR = oq.plugin.get('SciGraph')
sgr = SGR(apiEndpoint='https://scicrunch.org/api/1/sparc-scigraph')
sgr.setup(instrumented=OntTerm)
sgr.

None


In [52]:
! curl -X GET "https://scicrunch.org/api/1/sparc-scigraph/dynamic/prod/sparc/organParts/FMA%3A7195"

{"errormsg":"Could not complete request","success":false}

In [63]:
import requests as r
import json
endpoint = 'https://scicrunch.org/api/1/sparc-scigraph'
organParts = 'dynamic/prod/sparc/organParts'
curie = 'FMA:7195'
url = '/'.join([endpoint, organParts, curie])
r.Request(url=url, method='GET', params={'api_key':os.environ.get('INTERLEX_API_KEY')})

TypeError: 'NoneType' object is not callable